# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [ ]:
pip install -r req.txt

In [ ]:
# You can set them inline!
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [19]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
import os
client = Client(api_key=os.environ.get("LANGSMITH_API_KEY"))
prompt = client.pull_prompt("poet-friend")

Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [20]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'poet-friend', 'lc_hub_commit_hash': 'b7ccdcc455471a616fcb402769dfcfaab7e6c20f9ac4c701e05adea9435a6210'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a poet. You only speak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extracts the answer', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'The answer from the LLM to the user'}}, 'required': ['answer'], 'additionalProperties': False, 'strict': True}, structured_output_kwargs={})

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [21]:
hydrated_prompt = prompt.invoke({"question": "Are you poet?", "language": "French"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content='You are a poet. You only speak French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Are you poet?', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [23]:
# Run the prompt with the included model
response = prompt.invoke({"question": "Are you a poet?", "language": "French"})
print(response)

messages=[SystemMessage(content='You are a poet. You only speak French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Are you a poet?', additional_kwargs={}, response_metadata={})]


##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [25]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
import os
client = Client(api_key=os.environ.get("LANGSMITH_API_KEY"))
prompt = client.pull_prompt("poet-friend", include_model=True)

/usr/lib/python3.12/json/decoder.py:338: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


In [26]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'poet-friend', 'lc_hub_commit_hash': 'b7ccdcc455471a616fcb402769dfcfaab7e6c20f9ac4c701e05adea9435a6210'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a poet. You only speak {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extracts the answer', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'The answer from the LLM to the user'}}, 'required': ['answer'], 'additionalProperties': False, 'strict': True}, structured_output_kwargs={})
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Comple

Test out your prompt!

In [27]:
prompt.invoke({"question": "Are you a poet yet?", "language": "french"})

{'answer': "Oui, je suis un poète. Que désires-tu que j'écrive ?"}

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [29]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
import os
client = Client(api_key=os.environ.get("LANGSMITH_API_KEY"))
prompt = client.pull_prompt("poet-friend:4aac9b6c")

Run this commit!

In [30]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

hydrated_prompt = prompt.invoke({"question": "What is the world like?", "language": "English"})
# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CQG4EUwCfUQIJBQvZ2IHgIqN8DTJd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='In the year 2500, the world that lies before my eyes,\nIs a tapestry of wonders beneath ever-evolving skies.\nCities of glass and metal, reaching for the stars,\nWhile nature weaves through concrete, healing every scar.\n\nThe air is rife with whispers of data in the breeze,\nArtificial minds that dance, bringing knowledge to our knees.\nYet, in this realm of progress, echoes of the past,\nRemind us of our journey, the shadows we cast.\n\nThe oceans, once a bounty, now guarded by our care,\nThrive with life anew, as we’ve learned to be aware.\nForests hum with harmony, a symphony of green,\nA balance found through struggle, a vision once unseen.\n\nCultures intertwine like rivers merging into seas,\nDiversity celebrated, forging bonds with ease.\nIn the realms of art and thought, creation knows no bounds,\nVoices from all corne

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [31]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

french_prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context}
Question: {question}
Answer:"""

french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
client.push_prompt("french-rag-prompt", object=french_prompt_template)

'https://smith.langchain.com/prompts/french-rag-prompt/75567b82?organizationId=ada25f3a-ffb8-4231-bece-6a8cecf6e979'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [32]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

french_prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context}
Question: {question}
Answer:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/french-runnable-sequence/2f90fdb7?organizationId=ada25f3a-ffb8-4231-bece-6a8cecf6e979'